In [ ]:
import re
import time

start_time = time.time()

with open('/Users/warrickmoran/Desktop/jupyter/ncf-nwws-nesdis.20191121-cs2-ancf') as file:
    file_contents = file.read()
    print(file_contents)

In [ ]:
regex= r"^\s.END (\d{2}/\d{2}/\d{4}) (\d{2}:\d{2}:\d{2}) WMO\[((\S{4}\d{2}) (\S{4}) (\d{6})( \S{3}|\s{4}|\*\* no CR))\] (\{(\S{4,6})\s{0,2}?\} )?\#(\d+)/\d+ bytes\((\d+)\) c\(\d+\) .*/P(\d)[\n\r]*"

In [ ]:
match = re.findall(regex, file_contents, re.MULTILINE)
for item in match:
    print(item)

In [ ]:
import csv

with open('/Users/warrickmoran/Desktop/jupyter/ncf-nwws-nesdis.20191118-cs2-ancf.csv','w') as file:
    writer = csv.writer(file, lineterminator='\n')
    val = [['date','time','wmo','cccc','createtime','nnnxxx','size']]
    for item in match:
        val.append([item[0], item[1], item[2], item[4], item[5], item[8], item[10]])
    writer.writerows(val)


In [ ]:
from tinydb import TinyDB, Query
from tinydb.storages import MemoryStorage
db = TinyDB(storage=MemoryStorage)
products_table = db.table('products')
cccc_table = db.table('cccc')

#db = TinyDB('/home/warrick.moran/Downloads/drive-download-20200228T121647Z-001/db.json')
timedict = []
cccc = {}

for item in match: 
    timedict.append({'date':item[0],'time':item[1],'wmo':item[2],'cccc':item[4],'createtime':item[5],'nnnxxx':item[8],'size':item[10]})
    if item[4] not in cccc:
        cccc[item[4]] = 0
        cccc_table.insert({'cccc':item[4],'count':0})

products_table.insert_multiple(timedict)

print(products_table)

In [ ]:
from tinydb import where

results = cccc_table.all()
for result in results:
    count = products_table.count(where('cccc') == result['cccc'])
    cccc_table.update({'count':count}, where('cccc') == result['cccc'])
    
print(cccc_table)


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

x = [] 
y = []

xy = cccc_table.all()

for cccc in xy[0:10]:
    x.append(cccc['cccc'])
    y.append(cccc['count'])
    

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, y, color='green')
plt.xticks(x_pos, x)
plt.xlabel("NWS WFO WMO Identification")
plt.ylabel("Product Count")
plt.title("NWS Product Counts for NCF Uplink (20191121)")
plt.show()